# SPORTS PAGE EXTACTION WITHOUT STEMMING

This is not the main project, the main project is 206_Final_Project_ANSH.ipynb

In [1]:
import nltk
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import urllib
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from google.colab import drive
import matplotlib.pyplot as plt
% matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# TRAINING TO IDENTIFY SPORTS PAGES
folder = "https://cs-gitlab.union.edu/webbn/concordy/raw/master/Text/"
path = folder+'1970-10.txt'
req = Request(path)
year_text=''
try:
  urlopen(req)
except HTTPError as e:
  print(monthStr,'MISSING: Error code: ', e.code)
 
else:
  response = urllib.request.urlopen(path)
  data = response.read()      
  text = data.decode('utf-8') 
  year_text+=text
  pages = year_text.split('\n\n')
pages.pop(27)
pages.pop(41)
print ('No of pages:',len(pages))

tfidfVec = TfidfVectorizer(stop_words = 'english')
print (str(tfidfVec))
tfidf_matrix = tfidfVec.fit_transform(pages)

print ('Size of Vocabulary being used to train sports page cosine:',len(tfidfVec.vocabulary_))


No of pages: 47
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
Size of Vocabulary being used to train sports page cosine: 12438


In [ ]:
cos_similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
print('Cosine Similarity Matrix to a known sports page from volume (last page of last issue that month)\n\n',cos_similarity_matrix[-2])

Cosine Similarity Matrix to a known sports page from volume (last page of last issue that month)

 [0.05084457 0.0604494  0.05231469 0.06165406 0.05493235 0.02069139
 0.06006236 0.37529046 0.07093157 0.05327847 0.05967473 0.37913846
 0.04531017 0.04464335 0.05064203 0.3794788  0.07751888 0.05213985
 0.08147395 0.40480631 0.06394949 0.05564609 0.05997881 0.06110252
 0.03620397 0.04875734 0.05017815 0.4465135  0.05939377 0.06111053
 0.05522626 0.07000923 0.38348684 0.05288653 0.04208073 0.07870878
 0.46486959 0.05091817 0.06028207 0.05195156 0.42759463 0.03714651
 0.02872133 0.04450471 0.0667672  1.         0.        ]


In [ ]:
sports_docs = []
for page_no in range (0,len(cos_similarity_matrix[-2])):
  if cos_similarity_matrix[-2][page_no]>0.2:
    sports_docs+=[page_no]

print ('Pages from document pertaining to Oct 1970 issues that are sports related:', sports_docs)


Pages from document pertaining to Oct 1970 issues that are sports related: [7, 11, 15, 19, 27, 32, 36, 40, 45]


In [ ]:

sports_content = {1964:[]}
year = 1964
test_doc = [pages[-2]]
test_vector = tfidfVec.transform(test_doc)
test_cosine = (tfidf_matrix*test_vector.T).toarray()
test_sport_scores = []
for page_no in sports_docs:
  test_sport_scores+=[test_cosine[page_no]]
  print (test_cosine[page_no])



[0.37529046]
[0.37913846]
[0.3794788]
[0.40480631]
[0.4465135]
[0.38348684]
[0.46486959]
[0.42759463]
[1.]


In [ ]:

start_year = 1880
end_year = 2000
sports_content = {year:[] for year in range(start_year,end_year+1)}
print (sports_content)


{1880: [], 1881: [], 1882: [], 1883: [], 1884: [], 1885: [], 1886: [], 1887: [], 1888: [], 1889: [], 1890: [], 1891: [], 1892: [], 1893: [], 1894: [], 1895: [], 1896: [], 1897: [], 1898: [], 1899: [], 1900: [], 1901: [], 1902: [], 1903: [], 1904: [], 1905: [], 1906: [], 1907: [], 1908: [], 1909: [], 1910: [], 1911: [], 1912: [], 1913: [], 1914: [], 1915: [], 1916: [], 1917: [], 1918: [], 1919: [], 1920: [], 1921: [], 1922: [], 1923: [], 1924: [], 1925: [], 1926: [], 1927: [], 1928: [], 1929: [], 1930: [], 1931: [], 1932: [], 1933: [], 1934: [], 1935: [], 1936: [], 1937: [], 1938: [], 1939: [], 1940: [], 1941: [], 1942: [], 1943: [], 1944: [], 1945: [], 1946: [], 1947: [], 1948: [], 1949: [], 1950: [], 1951: [], 1952: [], 1953: [], 1954: [], 1955: [], 1956: [], 1957: [], 1958: [], 1959: [], 1960: [], 1961: [], 1962: [], 1963: [], 1964: [], 1965: [], 1966: [], 1967: [], 1968: [], 1969: [], 1970: [], 1971: [], 1972: [], 1973: [], 1974: [], 1975: [], 1976: [], 1977: [], 1978: [], 1979: [],

In [ ]:
def sports_page_collector():
  for year in sports_content:
    print ('Extracting from year:',year)
    for month in range(1,13):

      monthStr = str(month)

      if month < 10:
        monthStr = "0"+monthStr


      folder = "https://cs-gitlab.union.edu/webbn/concordy/raw/master/Text/"
      path = folder+str(year)+"-"+monthStr+".txt"
      req = Request(path)
      year_text=''
      try:
        urlopen(req)
      except HTTPError as e:
        print(monthStr,'MISSING: Error code: ', e.code)

      else:
        response = urllib.request.urlopen(path)
        data = response.read()      
        text = data.decode('utf-8') 
        year_text+=text
        pages = year_text.split('\n\n')    


        for page in pages:
          test_doc = [page]
          test_vector = tfidfVec.transform(test_doc)
          test_cosine = (tfidf_matrix*test_vector.T).toarray()
          test_sport_scores = []
          for page_no in sports_docs:
            test_sport_scores+=[test_cosine[page_no]]
            
          
          test_result = sum(test_sport_scores)/len(test_sport_scores)
          
          if test_result>0.15:
            sports_content[year]+=[page]
    print ('{} pages extracted from {}\n'.format(len(sports_content[year]),year))
%time sports_page_collector()

Extracting from year: 1880
07 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
0 pages extracted from 1880

Extracting from year: 1881
07 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
0 pages extracted from 1881

Extracting from year: 1882
07 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
2 pages extracted from 1882

Extracting from year: 1883
07 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
0 pages extracted from 1883

Extracting from year: 1884
06 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
0 pages extracted from 1884

Extracting from year: 1885
06 MISSING: Error code:  404
07 MISSING: Error code:  404
08 MISSING: Error code:  404
09 MISSING: Error code:  404
12 MISSING: Error code:  404
0 pages extracted from 1885

Extracting from year: 1886
07 MISSING: Error code:  404
08 MISSING: Error code:  404

In [ ]:
'''
drive.mount('/content/drive')
file = '/content/drive/My Drive/Colab Notebooks/concordy_sports_pages_yearwise1.pickle'
pickleOut = open(file,'wb')
pickle.dump(sports_content,pickleOut)
pickleOut.close()
'''

"\ndrive.mount('/content/drive')\nfile = '/content/drive/My Drive/Colab Notebooks/concordy_sports_pages_yearwise1.pickle'\npickleOut = open(file,'wb')\npickle.dump(sports_content,pickleOut)\npickleOut.close()\n"

In [ ]:
'''
drive.mount('/content/drive')

new_file = '/content/drive/My Drive/Colab Notebooks/concordy_sports_pages_yearwise_stemmed.pickle'
pickleIn = open(new_file,'rb')

sports_content=pickle.load(pickleIn)
'''

"\ndrive.mount('/content/drive')\n\nnew_file = '/content/drive/My Drive/Colab Notebooks/concordy_sports_pages_yearwise_stemmed.pickle'\npickleIn = open(new_file,'rb')\n\nsports_content=pickle.load(pickleIn)\n"

In [ ]:
print (sports_content[1970][0])

PAGE 6 JANUARY 9, 1970 Hot-Scoring Tedisco Breaks Marks; Hoopsters Face Harpur Tonight _ Despite two record~break.ing erformances by talented (i)phomore guard Jim Tedisco, the Union basketball team has managed only 1 win in its first 8 games. The Gamet were in action 5 times over winter recess, picking up their lone win over Drew, 93-78, while losing to Baldwin-Wallace, 81-6 7, Pace, 76-75 (in overtime), and Capital D·i st rid Tourney opponents Albany State and Siena. Sharpshooting Soph The brightest spot in Union's 69-70 campaign thus far must be the play of the 5-8 Tedisco. A graduate of Bishop Gibbons High School in Schenectady, he has been piling up points at nearly a 29 per game clip, while improving his play-making with every game. Tedisco poured in 44 points in the Dutchmen's one win, despite sitting out 10 minutes of play, to set ~ Union and Capital District colleges individual game _:lcoring mark. His 19 field goals SCHLOCK Anyone who wishes to p.lace a team in the I ndependen

In [ ]:
def content_len(sports_content):
  word_ctr=0
  page_ctr=0
  for year in sports_content:
    for page in sports_content[year]:
      word_ctr+=len(page.split())
      page_ctr+=1
  print ('No of total words:',word_ctr)
  print ('No of sports pages grabbed:',page_ctr)
  return ('\n')

print ('For the unstemmed iteration of sports content grabbing')
print ('\n',content_len(sports_content))

For the unstemmed iteration of sports content grabbing
No of total words: 8944463
No of sports pages grabbed: 4898

 

